# Importación de módulos generales

In [1]:
%pylab inline
%load_ext memory_profiler

# %pylab

import os
import tempfile
import pandas as pd
# import numpy as np
import networkx as nx
# import matplotlib
# import pylab  as plt
import pygraphviz

from pomegranate import BayesianNetwork

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

Populating the interactive namespace from numpy and matplotlib


# Funciones auxiliares visualización

Importamos las funciones auxiliares para visualizar redes que hemos definido:
- **plot_pomegranate_bn_nx**(pgm, layout=None, node_size=2000, node_color='pink')
- **plot_pomegranate_bn_pgvz**(pgm, filename=None, prog='dot', color='red')
- **plot_pgm_bn**(pgm, layout=None, node_size=2000, node_color='pink'):

In [2]:
from funciones_auxiliares import *

# Lectura de datos

Podemos obtener los datos originales de TODO.

In [3]:
X = pd.read_csv("data/fraud_detection/ucsd/DataminingContest2009.Task1.Train.Inputs")
y = pd.read_csv("data/fraud_detection/ucsd/DataminingContest2009.Task1.Train.Targets", header=None)
y.columns = ['target']
data = pd.merge(X, y, left_index=True, right_index=True)
X.head()

,amount,hour1,state1,zip1,field1,domain1,field2,hour2,flag1,total,field3,field4,field5,indicator1,indicator2,flag2,flag3,flag4,flag5
0,12.95,0,CA,925,3,AOL.COM,1,0,1,12.95,-4276,7,0,1,0,1,1,0,1
1,11.01,0,CA,925,3,AOL.COM,1,0,1,11.01,-4276,7,0,1,0,1,1,0,1
2,38.85,0,CA,928,3,HOTMAIL.COM,1,0,0,38.85,2602,21,1,0,0,0,0,0,1
3,25.90,0,NJ,77,0,AOL.COM,1,0,0,25.90,4139,6,0,0,0,1,1,0,1
4,12.95,0,CA,945,3,YAHOO.COM,0,0,1,12.95,3826,9,1,0,0,1,0,0,1


Vemos si hay valores nulos:

In [4]:
print(X.isnull().sum())

amount        0
hour1         0
state1        0
zip1          0
field1        0
domain1       1
field2        0
hour2         0
flag1         0
total         0
field3        0
field4        0
field5        0
indicator1    0
indicator2    0
flag2         0
flag3         0
flag4         0
flag5         0
dtype: int64


Encontramos el índice del valor nulo

In [5]:
null_index = X['domain1'].isnull().idxmax()
null_index

70382

Lo eliminamos tanto de X como de y:

In [6]:
X.dropna(inplace=True)
y.drop(null_index, inplace=True)
complete_data = data.dropna()

Vemos el número de valores únicos de cada variable

In [7]:
for col in list(X):
    print("{}: {}".format(col, X[col].nunique()))

amount: 88
hour1: 24
state1: 53
zip1: 899
field1: 5
domain1: 9809
field2: 2
hour2: 24
flag1: 2
total: 88
field3: 15786
field4: 38
field5: 25
indicator1: 2
indicator2: 2
flag2: 2
flag3: 2
flag4: 2
flag5: 36


Discretizamos las variables numéricas:

In [8]:
columns_to_discretize = ['amount', 'total', 'field3']
num_bins=[20,20,100]

for col,n in zip(columns_to_discretize, num_bins):
    X[col] = pd.cut(X[col], bins=n)
    
X.head()

,amount,hour1,state1,zip1,field1,domain1,field2,hour2,flag1,total,field3,field4,field5,indicator1,indicator2,flag2,flag3,flag4,flag5
0,"(9.54, 14.31]",0,CA,925,3,AOL.COM,1,0,1,"(9.54, 14.31]","(-4348.98, -3944.4]",7,0,1,0,1,1,0,1
1,"(9.54, 14.31]",0,CA,925,3,AOL.COM,1,0,1,"(9.54, 14.31]","(-4348.98, -3944.4]",7,0,1,0,1,1,0,1
2,"(38.16, 42.93]",0,CA,928,3,HOTMAIL.COM,1,0,0,"(38.16, 42.93]","(2528.88, 2933.46]",21,1,0,0,0,0,0,1
3,"(23.85, 28.62]",0,NJ,77,0,AOL.COM,1,0,0,"(23.85, 28.62]","(3742.62, 4147.2]",6,0,0,0,1,1,0,1
4,"(9.54, 14.31]",0,CA,945,3,YAHOO.COM,0,0,1,"(9.54, 14.31]","(3742.62, 4147.2]",9,1,0,0,1,0,0,1


Dividimos los datos en conjunto de entrenamiento y validación

In [9]:
X_complete = X
y_complete = y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)
X_complete_train, X_complete_test, y_complete_train, y_complete_test = train_test_split(X_complete, y_complete, test_size=1/3)

print("Particiones obtenidas sobre los datos iniciales:")
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test:  ", X_test.shape)
print("y_test:  ", y_test.shape)
print("\nParticiones obtenidas sobre los datos con los registros completos:")
print("X_complete_train: ", X_complete_train.shape)
print("y_complete_train: ", y_complete_train.shape)
print("X_complete_test:  ", X_complete_test.shape)
print("y_complete_test:  ", y_complete_test.shape)

Particiones obtenidas sobre los datos iniciales:
X_train:  (63120, 19)
y_train:  (63120, 1)
X_test:   (31561, 19)
y_test:   (31561, 1)

Particiones obtenidas sobre los datos con los registros completos:
X_complete_train:  (63120, 19)
y_complete_train:  (63120, 1)
X_complete_test:   (31561, 19)
y_complete_test:   (31561, 1)


# Algoritmos de búsqueda global

In [10]:
from pgmpy.models import BayesianModel

Definición de scores sobre el dataset:

In [11]:
from pgmpy.estimators import BDeuScore, K2Score, BicScore

bdeu = BDeuScore(complete_data, equivalent_sample_size=5)
k2 = K2Score(complete_data)
bic = BicScore(complete_data)

Si quisieramos usar un algoritmo exacto, tendriamos que iterar sobre los $2^{N(N-1)}$ siendo N el número de nodos, es decir:

In [12]:
N=len(list(complete_data))
print("Número de DAGs posibles con N={} ---> {}".format(N, 2**(N*(N-1))))

Número de DAGs posibles con N=20 ---> 2462625387274654950767440006258975862817483704404090416746768337765357610718575663213391640930307227550414249394176


In [13]:
# from pgmpy.estimators import ExhaustiveSearch

# # Definición algoritmos de búsqueda exactos con cada score
# es_bic = ExhaustiveSearch(complete_data, scoring_method=bic)
# es_k2 = ExhaustiveSearch(complete_data, scoring_method=k2)
# es_bdeu = ExhaustiveSearch(complete_data, scoring_method=bdeu)

# # Aprendizaje de las redes
# print("Exhaustive search BIC:")
# %time %memit es_bic_structure = es_bic.estimate()
# print("Exhaustive search K2:")
# %time %memit es_k2_structurel = es_k2.estimate()
# print("Exhaustive search BDeu:")
# %time %memit es_bdeu_structure = es_bdeu.estimate()

Así que vamos a usar Hill Climb Search:

In [ ]:
from pgmpy.estimators import HillClimbSearch

hc_bic = HillClimbSearch(data, scoring_method=bic)
hc_k2 = HillClimbSearch(data, scoring_method=k2)
hc_bdeu = HillClimbSearch(data, scoring_method=bdeu)

# Aprendizaje de las redes
print("Hill Climb Search BIC:")
%time %memit hc_bic_structure = hc_bic.estimate(max_indegree=1)
# %time %memit hc_bic_structure = hc_bic.estimate()
# print("\nHill Climb Search K2:")
# %time %memit hc_k2_structure = hc_k2.estimate()
print("\nHill Climb Search BDeu:")
%time %memit hc_bdeu_structure = hc_bdeu.estimate()

Hill Climb Search BIC:


Visualizamos los modelos resultantes:

In [ ]:
print("Hill Climb Search BIC:")
plot_pgmpy_bn(hc_bic_structure)

In [ ]:
# print("\nHill Climb Search K2:")
# plot_pgmpy_bn(hc_k2_structure)

In [ ]:
print("\nHill Climb Search BDeu:")
plot_pgmpy_bn(hc_bdeu_structure)

A la hora de estimar, podemos indicar el número máximo de padres por variable. Por ejemplo, si exigimos que sea 1, aprenderemos estructuras de árbol y el aprendizaje será más rápido:

In [ ]:
print("Hill Climb Search BIC:")
%time %memit hc_bic_tree = hc_bic.estimate(max_indegree=1)
# print("\nHill Climb Search K2:")
# %time %memit hc_k2_tree = hc_k2.estimate(max_indegree=1)
print("\nHill Climb Search BDeu:")
%time %memit hc_bdeu_tree = hc_bdeu.estimate(max_indegree=1)

In [ ]:
print("Hill Climb Search BIC tree:")
plot_pgmpy_bn(hc_bic_tree)

In [ ]:
# print("\nHill Climb Search K2 tree:")
# plot_pgmpy_bn(hc_k2_tree)

In [ ]:
print("\nHill Climb Search BDeu tree:")
plot_pgmpy_bn(hc_bdeu_tree)

# Algoritmos locales mediante tests de independencia

In [ ]:
from pgmpy.estimators import ConstraintBasedEstimator

constraint_estimator = ConstraintBasedEstimator(data)

print("Constraint Estimation:")
%time %memit constraint_structure = constraint_estimator.estimate(significance_level=0.01)

In [ ]:
print("Constraint Estimation:")
plot_pgmpy_bn(constraint_structure)

# Aprendizaje de parámetros

Vamos a quedarnos, por ejemplo, con la red aprendida usando *Hill Climb Search* y como score *Bdeu*

In [ ]:
from pgmpy.models import BayesianModel

# hc_bdeu_model = BayesianModel()
# hc_bdeu_model.add_nodes_from(hc_bdeu_structure.nodes)
# hc_bdeu_model.add_edges_from(hc_bdeu_structure.edges)

Podemos obtener el recuento de las apariciones de las distintas variables

In [ ]:
from pgmpy.estimators import ParameterEstimator

# pe = ParameterEstimator(hc_bdeu_model, data)
# print(pe.state_counts('Age'))

O directamente las probabilidades usando un estimador máximo verosimil, que en este caso se corresponde a las frecuencias relativas

In [ ]:
# from pgmpy.estimators import MaximumLikelihoodEstimator
# mle = MaximumLikelihoodEstimator(hc_bdeu_model, data)
# print(mle.estimate_cpd('Age')) 

Para obtener todas las TPC a la vez:

In [ ]:
# hc_bdeu_model.fit(data, estimator=MaximumLikelihoodEstimator)

Podemos usar estimadores bayesianos, suponiendo una distribucióna  priori:

In [ ]:
# from pgmpy.estimators import BayesianEstimator

# est = BayesianEstimator(hc_bdeu_model, data)
# print(est.estimate_cpd('Age', prior_type='BDeu', equivalent_sample_size=10))

# hc_bdeu_model.fit(data, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5

Vemos como las probabilidades que antes eran 0 ahora no lo son

# Comparación accuracy distintas estructuras

Creamos los modelos a partir de las estructuras:

In [ ]:
from pgmpy.models import BayesianModel

hc_bdeu_model = BayesianModel()
hc_bdeu_model.add_nodes_from(hc_bdeu_structure.nodes)
hc_bdeu_model.add_edges_from(hc_bdeu_structure.edges)

# hc_k2_model = BayesianModel()
# hc_k2_model.add_nodes_from(hc_k2_structure.nodes)
# hc_k2_model.add_edges_from(hc_k2_structure.edges)

hc_bic_model = BayesianModel()
hc_bic_model.add_nodes_from(hc_bic_structure.nodes)
hc_bic_model.add_edges_from(hc_bic_structure.edges)

constraint_model = BayesianModel()
constraint_model.add_nodes_from(constraint_structure.nodes)
constraint_model.add_edges_from(constraint_structure.edges)

Estimamos los parámetros usando un estimador bayesiano con priori uniforme:

In [ ]:
from pgmpy.estimators import BayesianEstimator

hc_bdeu_model.fit(data, estimator=BayesianEstimator, prior_type="BDeu")
# hc_k2_model.fit(data, estimator=BayesianEstimator, prior_type="BDeu")
hc_bic_model.fit(data, estimator=BayesianEstimator, prior_type="BDeu")
constraint_model.fit(data, estimator=BayesianEstimator, prior_type="BDeu")

Preparamos la inferencia con eliminación de variables:

In [ ]:
# from pgmpy.inference import VariableElimination

# hc_bdeu_inference = VariableElimination(hc_bdeu_model)
# hc_k2_inference = VariableElimination(hc_k2_model)
# hc_bic_inference = VariableElimination(hc_bic_model)
# constraint_inference = VariableElimination(constraint_model)

Ejecutamos la inferencia:

In [ ]:
# X_complete_train_dict = list(X_complete_train.to_dict('index').values())

# %time %memit y_pred_hc_bdeu = [hc_bdeu_inference.map_query(variables=['Outcome'], evidence=entry, elimination_order='MinFill', \
#                     show_progress=False)['Outcome'] for entry in X_complete_train_dict]
# %time %memit y_pred_hc_k2 = [hc_k2_inference.map_query(variables=['Outcome'], evidence=entry, elimination_order='MinFill', \
#                     show_progress=False)['Outcome'] for entry in X_complete_train_dict]
# %time %memit y_pred_hc_bic = [hc_bic_inference.map_query(variables=['Outcome'], evidence=entry, elimination_order='MinFill', \
#                     show_progress=False)['Outcome'] for entry in X_complete_train_dict]
# %time %memit y_pred_constraint = [constraint_inference.map_query(variables=['Outcome'], evidence=entry, elimination_order='MinFill', \
#                         show_progress=False)['Outcome'] for entry in X_complete_train_dict]

# # for entry in X_complete_train_dict:
#     print(hc_bdeu_inference.map_query(variables=['Outcome'], evidence=entry, elimination_order='MinFill')['Outcome'])
# hc_bdeu_inference.map_query(variables=['Outcome'], evidence=None, elimination_order='MinFill')

O más facil:

In [ ]:
%time %memit y_pred_hc_bdeu = hc_bdeu_model.predict(X_complete_train)
%time %memit y_pred_hc_k2 = hc_k2_model.predict(X_complete_train)
%time %memit y_pred_hc_bic = hc_bic_model.predict(X_complete_train)
%time %memit y_pred_constraint = constraint_model.predict(X_complete_train)

In [ ]:
print("HC BDeu:\n", classification_report(list(y_pred_hc_bdeu['target']), list(y_complete_train)))
print("HC K2:\n", classification_report(list(y_pred_hc_k2['target']), list(y_complete_train)))
print("HC BIC:\n", classification_report(list(y_pred_hc_bic['target']), list(y_complete_train)))
print("Constraint:\n", classification_report(list(y_pred_constraint['target']), list(y_complete_train)))

Sobre los datos de validación:

In [ ]:
%time %memit y_pred_hc_bdeu = hc_bdeu_model.predict(X_complete_test)
%time %memit y_pred_hc_k2 = hc_k2_model.predict(X_complete_test)
%time %memit y_pred_hc_bic = hc_bic_model.predict(X_complete_test)
%time %memit y_pred_constraint = constraint_model.predict(X_complete_test)

In [ ]:
print("HC BDeu:\n", classification_report(list(y_pred_hc_bdeu['target']), list(y_complete_test)))
print("HC K2:\n", classification_report(list(y_pred_hc_k2['target']), list(y_complete_test)))
print("HC BIC:\n", classification_report(list(y_pred_hc_bic['target']), list(y_complete_test)))
print("Constraint:\n", classification_report(list(y_pred_constraint['target']), list(y_complete_test)))

O directamente sobre el dataset completo

In [ ]:
X_complete_data = complete_data.drop(columns='target')
y_complete_data = complete_data['target']

%time %memit y_pred_hc_bdeu = hc_bdeu_model.predict(X_complete_data)
%time %memit y_pred_hc_k2 = hc_k2_model.predict(X_complete_data)
%time %memit y_pred_hc_bic = hc_bic_model.predict(X_complete_data)
%time %memit y_pred_constraint = constraint_model.predict(X_complete_data)

In [ ]:
print("HC BDeu:\n", classification_report(list(y_pred_hc_bdeu['target']), list(y_complete_data)))
print("HC K2:\n", classification_report(list(y_pred_hc_k2['target']), list(y_complete_data)))
print("HC BIC:\n", classification_report(list(y_pred_hc_bic['target']), list(y_complete_data)))
print("Constraint:\n", classification_report(list(y_pred_constraint['target']), list(y_complete_data)))